# Geodetic Data Exercise

## Exercise A: Get Geodetic Data with the EarthScope SDK

Using the EarthScope SDK, stream data GNSS observations data to a Pandas dataframe and show the observables.

In [ ]:
# --- Fill in the blanks (marked ____ ) ---
import datetime as dt
import pandas as pd
from earthscope_sdk import EarthScopeClient

# 1) Create the client
es = ____()

# 2) Define a start and end datetime (UTC)
start_dt = dt.datetime(____, ____, ____, ____)  # year, month, day, hour
end_dt   = dt.datetime(____, ____, ____, ____)  # ~30 hours later

# 3) Choose a station (4-character ID)
station = "____"

# 4) Make the request for GNSS observations and fetch as an Arrow table
arrow_table = es.data.gnss_observations(
    start_datetime=____,
    end_datetime=____,
    station_name=____,
).____()  # method that actually runs the request

# 5) Convert Arrow ➜ pandas DataFrame and view the first few rows
df = arrow_table.____()
print(df.head())

# 6) (bonus) How many rows did we get?
print("rows:", len(df))

# 7) (bonus) Show unique observation types if present (e.g., L1, L2, C1...)
if "observable" in df.columns:
    print("observables:", sorted(df["observable"].unique()))

## Exercise B: Download a RINEX file

In [ ]:
# ---------- Fill in the package imports for this script ----------
import ____  
from datetime import datetime
from pathlib import Path
import ____

# ---------- Create an EarthScope client to get token ----------
client = ____


# Fill this with the service endpoint for GAGE data
GAGE_URL = ____

DATA_DIR = Path("./data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# ---------- Get an EarthScope token ----------
def get_token():
    """
    Use the SDK to refresh (if needed) and return an access token.
    """
    # Refresh if necessary
    ____

    # Retrieve the token string
    token = ____
    return token


# ---------- Helper: common auth header ----------
def auth_header(token):
    """
    Helper function to create header with token
    """
    # Use a standard Bearer token header (lowercase 'authorization' is fine)
    return ____


# ---------- Download a file from EarthScope's web services ----------
# --------------------------------------------------------------------
def download_data(url, data_directory):  # params is an optional query parameter
    """
    Sends GET request to GAGE web service by construcing the URL to the RINEX file.
    """
    # get authorization token
    token = get_token()

    # create the authorization header 
    header = auth_header(token) 


    # This creates a file name for a RINEX file.
    file_name = Path(url).name

    # create a variable with the path to the data directory and file
    out_path = Path(Path(data_directory) / ____)

    # Make the request to the web service with params and bearer auth
    r = requests.____(
        url,
        params=____,
        headers=____,
        stream=True
    ) 

    if r.status_code == requests.codes.____:
        with open(out_path, "wb") as f:
            for data in r:
                f.write(data)
    else:
        #problem occured
        print(f"failure: {r.status_code}, {r.reason}")
        return None


# ---------- Creates URL to download data from the EarthScope GAGE web service ----------
# ---------------------------------------------------------------------------------------
def create_url(year, day, station, compression):
    """
    Construct a URL to a file in an archive given year, day-of-year, station, etc.
    Example output:
      {BASE_URL}{year}/{DOY}/{/STATIONDOY0.YY}{compression}
    """
    doy =  "{:03d}".____(day)
    two_digit_year = ____  # Hint: use the string slice function in the previous notebook

    file_path = "/".join([str(year), doy])
    file_name = "".join(["/", station, doy, "0.", two_digit_year, compression])
    url = "".join([GAGE_URL, file_path, file_name])
    return url

# Download a RINEX file
#
year = 2025
day = 1
station = 'p034'
doy = '%03d'.format(day)
compression = 'd.Z'  # or ".Z" / "" depending on the archive
url = create_url(year, ____, station, compression)
download_data(url, ____)

## Answer Key for Exercise A

```{admonition} Click to see answer
:class: dropdown

<PRE>
import datetime as dt
import pandas as pd
from earthscope_sdk import EarthScopeClient

es = EarthScopeClient()

start_dt = dt.datetime(2025, 7, 20, 21)
end_dt   = dt.datetime(2025, 7, 22, 3)
station = "AC60"

arrow_table = es.data.gnss_observations(
    start_datetime=start_dt,
    end_datetime=end_dt,
    station_name=station,
).fetch()

df = arrow_table.to_pandas()
print(df.head())
print("rows:", len(df))

if "observable" in df.columns:
    print("observables:", sorted(df["observable"].unique()))

</PRE>
```


## Answer Key for Exercise B

```{admonition} Click to see answer
:class: dropdown

<PRE>
# ---------- Imports ----------
import requests
import os
from datetime import datetime
from pathlib import Path
from earthscope_sdk import EarthScopeClient

# ---------- Create an EarthScope client to get token ----------
client = EarthScopeClient()

# Fill this with the service endpoint for SAGE data
GAGE_URL = "https://gage-data.earthscope.org/archive/gnss/rinex/obs/?"

# create a directory for RINEX data
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

# ---------- Get an EarthScope token ----------
def get_token():
    """
    Use the SDK to refresh (if needed) and return an access token.
    """
    # Refresh if necessary
    client.ctx.auth_flow.refresh_if_necessary()

    # Retrieve the token string
    token = client.ctx.auth_flow.access_token
    return token


# ---------- Helper: common auth header ----------
def auth_header(token):
    # Use a standard Bearer token header (lowercase 'authorization' is fine)
    return {"authorization": f"Bearer {token}"}


# ---------- Download a file from EarthScope's web services ----------
# --------------------------------------------------------------------
def download_data(url, data_directory):
    """
    Sends GET with query parameters and saves the response body to a file.
    """
    # get authorization token
    token = get_token()

    # create the authorization header 
    header = auth_header(token) 
    
    # Create file name 
    file_name = Path(url).name

    out_path = Path(Path(data_directory) / file_name)

    # Make the request with params and bearer auth
    r = requests.get(
        url,
        params=params,
        headers=header,
        stream=True
    ) 

    if r.status_code == requests.codes.ok:
        with open(out_path, "wb") as f:
            for data in r:
                f.write(data)
    else:
        #problem occurred
        print(f"failure: {r.status_code}, {r.reason}")
        return None

# Try the URL-building flow
year = 2025
day = 1
station = 'p034'
doy = '%03d'.format(day)
compression = 'd.Z'  # or ".Z" / "" depending on the archive
url = create_url(year, day, station, compression)
download_data(url, DATA_DIR)
</PRE>
```

## [< Previous](./6_geodetic_data.ipynb)&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;[Next >](./8_wrap_up.ipynb)